In [1]:
%pylab
%matplotlib inline

import cv2
import subprocess
from segmetrics.study    import *
from segmetrics.regional import *
from segmetrics.boundary import *
from skimage import measure
from tempfile import NamedTemporaryFile

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [24]:
def load_xcf_layer(xcf_path, layername):
    with NamedTemporaryFile() as png_file:
        subprocess.call(['xcf2png', xcf_path, layername, '-o', png_file.name])
        img = cv2.imread(png_file.name, 0)
        if img is None: warnings.warn('couldn\'t load layer')
        return img

def load_regions(xcf_path):
    foreground = load_xcf_layer(xcf_path, 'foreground')
    regions = measure.label(foreground, neighbors=4, background=0)
    assert all((regions == -1) == (foreground == 0))
    return regions + 1

In [25]:
result      = load_regions('testdata/result.xcf')
groundtruth = load_regions('testdata/groundtruth.xcf')

In [20]:
study = Study()
study.add_measure(Dice()     , 'dice')
study.add_measure(Hausdorff(), 'hausdorff')
study.add_measure(Recall()   , 'recall')
study.add_measure(Precision(), 'precision')
study.add_measure(Accuracy() , 'accuracy')
study.process(result, groundtruth)
for measure_name in study.results: study.results[measure_name] = mean(study.results[measure_name])
study.results

{'accuracy': 0.30415177419006345,
 'dice': 0.91634550767453693,
 'hausdorff': 16.480058084071327,
 'precision': 0.9860045835104515,
 'recall': 0.85587948646990175}